In [1]:
# 导入NCS—SK—API
from mvnc import mvncapi as mvnc
# 导入facenet模块包
import facenet_ncs
import cv2
import sys
import os
import numpy as np
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from time import sleep
import realtime_input_HDMI

# 加载.bit文件
base = BaseOverlay("base.bit")

In [2]:
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# Configure the HDMI output to the same resolution as the HDMI input
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

# Start the HDMI interfaces
hdmi_in.start()
hdmi_out.start()

In [3]:
# ！！！facenet模型文件路径 ——— facenet.graph ！！！
GRAPH_FILENAME = "/home/xilinx/jupyter_notebooks/ncs-facenet_tensorflow/models/facenet.graph"
# 人脸录入路径
targets_list_dir = '/home/xilinx/jupyter_notebooks/ncs-facenet_tensorflow/targets'

In [4]:
# 检查NCS
devices = mvnc.EnumerateDevices()
if len(devices) == 0:
    print('No NCS devices found')
    quit()
# 开启NCS
device = mvnc.Device(devices[0])
device.OpenDevice()
# 准备模型文件(加载至片上内存)——facenet.graph
graph_file_name = GRAPH_FILENAME
with open(graph_file_name, mode='rb') as f:
    graph_in_memory = f.read()
graph = device.AllocateGraph(graph_in_memory)
# 获取人脸目标路径
temp_list = {}
targets_list = os.listdir(targets_list_dir)
targets_list = [i for i in targets_list if i.endswith('.jpg')]
# ！！！人脸录入 ！！！（获取目标人脸特征集）
targets_feature = facenet_ncs.feature(targets_list, temp_list, graph)
print('!!!ok ok ok!!!')

!!!ok ok ok!!!


In [5]:
# 实时循环识别人脸
Delay = 0.5
i = 0
while True:
    # 读入一帧图像
    frame = hdmi_in.readframe()
    
    if (base.buttons[0].read()==1):
        sleep(Delay)
        if (base.buttons[0].read()==1):
            for led in base.leds:
                led.toggle()
            targets_feature, targets_list = realtime_input_HDMI.realtime_input(frame, hdmi_in, hdmi_out, graph)
   
    i = i + 1
    # 每隔2帧处理1帧图像
    if(i > 2):
        i = 0
        # ！！！人脸识别 ！！！
        frame_res = facenet_ncs.run_images(targets_feature, targets_list, graph, frame)   
        
        # HDMI输出
        frame_out = hdmi_out.newframe()
          # ！图像由ARM——>FPGA！
        frame_out[:,:,:] = frame_res[:,:,:]
        hdmi_out.writeframe(frame_out)

Total Difference is: 0.7633564472198486
Total Difference is: 0.7633564472198486
Total Difference is: 0.7601682543754578
Total Difference is: 0.759426474571228
Total Difference is: 0.759426474571228
Total Difference is: 0.7597525715827942
Total Difference is: 0.7636825442314148
Total Difference is: 0.7656230330467224
Total Difference is: 0.7656230330467224
Total Difference is: 0.7664594054222107
Total Difference is: 0.7640589475631714
Total Difference is: 0.761383593082428
Total Difference is: 0.761383593082428
Total Difference is: 0.761383593082428
Total Difference is: 0.7634855508804321
Total Difference is: 0.7677298784255981
Total Difference is: 0.7676381468772888
Total Difference is: 0.7676381468772888
Total Difference is: 0.7676381468772888
Total Difference is: 0.7658490538597107
Total Difference is: 0.764954149723053
Total Difference is: 0.7758092284202576
Total Difference is: 0.7634611129760742
Total Difference is: 0.7533577680587769
Total Difference is: 0.7649511098861694
Total 

Total Difference is: 1.229511559009552
Total Difference is: 1.0269370675086975
Total Difference is: 1.1414076089859009
Total Difference is: 1.8243250250816345
Total Difference is: 1.9490002989768982
Total Difference is: 1.3308714032173157
Total Difference is: 0.927079975605011
Total Difference is: 0.8219091296195984
Total Difference is: 0.8930020928382874
Total Difference is: 0.8518903851509094
Total Difference is: 0.849344789981842
Total Difference is: 0.8501341938972473
Total Difference is: 0.8831741809844971
Total Difference is: 0.8876861333847046
Total Difference is: 0.8603625297546387
Total Difference is: 0.8655737638473511
Total Difference is: 0.9058662056922913
Total Difference is: 0.9005379676818848
Total Difference is: 1.0651840567588806
Total Difference is: 1.061303973197937
Total Difference is: 1.0913089513778687
Total Difference is: 1.0739762783050537
Total Difference is: 1.0584312677383423
Total Difference is: 0.8728306293487549
Total Difference is: 0.8754705786705017
Tota

KeyboardInterrupt: 